# Для тренировки модели нужно установить turicreate
Linux or mac
python 3.6 - 3.8

`pip install turicreate`

### На вход:
- _attend_features.csv (с нашими фичами)
- _groups.csv

### Выход:
- predicts/predict_v01.csv


In [1]:
import turicreate as tc
import pandas as pd

In [2]:
actions = tc.SFrame.read_csv('_attend_features.csv')

Finished parsing file /media/vasiliev/SSD_Data/_dev/ezee_may_2003/ds/_attend_features.csv

Parsing completed. Parsed 100 lines in 0.215888 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,int,float,str,str,str,float,str,str,str,str,float,float,str,int,str,float,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 173459 lines. Lines per second: 617162

Finished parsing file /media/vasiliev/SSD_Data/_dev/ezee_may_2003/ds/_attend_features.csv

Parsing completed. Parsed 600390 lines in 0.905295 secs.

### Делим общую таблицу на фичи для attend, users, groups

In [3]:
print(actions.head)
actions1 = actions[['group_id',
                    'user_id',
                    'log_visits',
                    #'user_lat',
                    #'user_long',
                    'направление 1',
                    'направление 2',
                    #'направление 3',
                    'weekend']]

users_1 = actions[['user_id',
                   'sex','age',
                   'user_address',
                   'user_lat',
                   'user_long']]

groups_1 = actions[['group_id',
                    'lat',
                    'long',
                    'направление 1',
                    'направление 2',
                    'направление 3',
                    'is_online',
                    'weekend',
                    'lesson_length',
                    'session']]

<bound method SFrame.head of Columns:
	X1	int
	group_id	int
	user_id	int
	log_visits	float
	направление 3	str
	is_online	str
	weekend	str
	lesson_length	float
	session	str
	направление 1	str
	направление 2	str
	group_address	str
	lat	float
	long	float
	sex	str
	age	int
	user_address	str
	user_lat	float
	user_long	float

Rows: 600390

Data:
+----+-----------+-----------+--------------------+---------------+-----------+
| X1 |  group_id |  user_id  |     log_visits     | направление 3 | is_online |
+----+-----------+-----------+--------------------+---------------+-----------+
| 0  | 801346549 | 101370217 | 2.0794415416798357 |  Здорово жить |  offline  |
| 1  | 801346549 | 101375025 | 3.044522437723423  |  Здорово жить |  offline  |
| 2  | 801346549 | 101392971 |        0.0         |  Здорово жить |  offline  |
| 3  | 801346550 | 101352023 | 4.1588830833596715 |  ОНЛАЙН Цигун |   online  |
| 4  | 801346550 | 101354499 | 4.1588830833596715 |  ОНЛАЙН Цигун |   online  |
| 5  | 801346550 |

There are a variety of machine learning techniques that can be used to build a recommender model. Turi Create provides a method turicreate.recommender.create that will automatically choose an appropriate model for your data set. More details on choosing a model: https://apple.github.io/turicreate/docs/userguide/recommender/choosing-a-model.html

First we create a random split of the data to produce a validation set that can be used to evaluate the model.

### Чему учим модель

Мы учим модель предсказывать логарифм от количества посещений юзером курса (логарифм потому что разброс посещений курса пользователем высокий - от 1 до 64)

### Создание и обучение дефолтной модели

In [4]:
training_data, validation_data = tc.recommender.util.random_split_by_user(actions1, 'user_id', 'group_id', item_test_proportion = 0.01)

model = tc.recommender.create(training_data,
                              'user_id',
                              'group_id',
                              target = 'log_visits',
                              user_data = users_1,
                              item_data = groups_1
                             )


Preparing data set.

Data has 600249 observations with 50906 users and 24535 items.

Data prepared in: 0.719134s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.48189

Final training RMSE: 0.577227

In [5]:
# параметры модели
model

Class                            : RankingFactorizationRecommender

Schema
------
User ID                          : user_id
Item ID                          : group_id
Target                           : log_visits
Additional observation features  : 3
User side features               : ['user_id', 'sex', 'age', 'user_address', 'user_lat', 'user_long']
Item side features               : ['group_id', 'lat', 'long', 'направление 1', 'направление 2', 'направление 3', 'is_online', 'weekend', 'lesson_length', 'session']

Statistics
----------
Number of observations           : 600249
Number of users                  : 50906
Number of items                  : 24535

Training summary
----------------
Training time                    : 16.3295

Model Parameters
----------------
Model class                      : RankingFactorizationRecommender
num_factors                      : 32
binary_target                    : 0
side_data_factorization          : 1
solver                           : auto
n

### Получение предсказаний от дефолтной модели

In [6]:
default_recommendations = model.recommend()

recommendations finished on 1000/50906 queries. users per second: 846.034

recommendations finished on 2000/50906 queries. users per second: 848.58

recommendations finished on 3000/50906 queries. users per second: 851.569

recommendations finished on 4000/50906 queries. users per second: 852.843

recommendations finished on 5000/50906 queries. users per second: 853.684

...

recommendations finished on 46000/50906 queries. users per second: 853.077

recommendations finished on 47000/50906 queries. users per second: 834.637

recommendations finished on 48000/50906 queries. users per second: 813.178

recommendations finished on 49000/50906 queries. users per second: 793.124

recommendations finished on 50000/50906 queries. users per second: 774.359

### Посмотрим рекоммендации для отдельных юзеров

In [7]:
df_groups = pd.read_csv('groups.csv')

def display_history_and_predictions_for_user(user_id, train_dataset, predictions):
    print('----------------------------------------------------------------------------')
    print('User', user_id, 'history:')
    display(train_dataset[train_dataset['user_id'] == user_id][
        'group_id','is_online','session','направление 1','направление 2'])
    predictions = predictions[predictions['user_id'] == user_id].to_dataframe()
    print('User recommendations:')
    predicted_groups = predictions['group_id'].to_list()
    display(df_groups[df_groups['уникальный номер'].isin(predicted_groups)].drop(columns=[
        'адрес площадки','расписание в активных периодах','расписание в закрытых периодах','расписание в плановом периоде']))

In [8]:
user_id = 101352882

display_history_and_predictions_for_user(user_id, actions, default_recommendations)

----------------------------------------------------------------------------
User 101352882 history:


group_id,is_online,session,направление 1,направление 2
801348234,online,mornig,Физическая активность,ОНЛАЙН Гимнастика
801348838,offline,mornig,Физическая активность,Гимнастика
801348944,offline,mornig,Танцы,Танцы
801349048,offline,mornig,Физическая активность,ОФП
801350227,online,mornig,Творчество,ОНЛАЙН Художественно-прикладное творчество ...
801350380,online,mornig,Танцы,ОНЛАЙН Танцы
801351413,online,mornig,Рисование,ОНЛАЙН Рисование
801358464,online,mornig,Физическая активность,ОНЛАЙН Гимнастика
801358465,online,mornig,Танцы,ОНЛАЙН Танцы
801358466,online,mornig,Физическая активность,ОНЛАЙН Гимнастика


User recommendations:


,уникальный номер,направление 1,направление 2,направление 3,округ площадки,район площадки
1008,801354484,Рисование,ОНЛАЙН Рисование,ОНЛАЙН ИЗО,Юго-Восточный административный округ,муниципальный округ Люблино
1093,801346701,Рисование,ОНЛАЙН Рисование,ОНЛАЙН Масляная живопись,Восточный административный округ,муниципальный округ Северное Измайлово
1840,801352851,Рисование,ОНЛАЙН Рисование,ОНЛАЙН Китайская живопись У-Син,Восточный административный округ,муниципальный округ Косино-Ухтомский
1841,801348029,Рисование,ОНЛАЙН Рисование,ОНЛАЙН Китайская живопись У-Син,Юго-Восточный административный округ,муниципальный округ Некрасовка
2920,801350775,Рисование,ОНЛАЙН Рисование,ОНЛАЙН Китайская живопись У-Син,NaN,NaN
2924,801352907,Рисование,ОНЛАЙН Рисование,ОНЛАЙН Китайская живопись У-Син,NaN,NaN
2927,801352685,Рисование,ОНЛАЙН Рисование,ОНЛАЙН Различные техники рисования,NaN,NaN
3752,801347074,Физическая активность,ОНЛАЙН Гимнастика,ОНЛАЙН Корригирующая гимнастика для глаз,NaN,NaN
3915,801348042,Рисование,ОНЛАЙН Рисование,ОНЛАЙН Масляная живопись,Зеленоградский административный округ,муниципальный округ Крюково
3971,801350767,Рисование,ОНЛАЙН Рисование,ОНЛАЙН Различные техники рисования,NaN,NaN


In [9]:
user_id = 101354253

display_history_and_predictions_for_user(user_id, actions, default_recommendations)

----------------------------------------------------------------------------
User 101354253 history:


group_id,is_online,session,направление 1,направление 2
801347802,online,mornig,Физическая активность,ОНЛАЙН ОФП
801348667,online,evening,Физическая активность,ОНЛАЙН Гимнастика
801348794,online,mornig,Физическая активность,ОНЛАЙН Гимнастика
801349796,offline,mornig,Физическая активность,Гимнастика
801349875,offline,aftennoon,Физическая активность,Гимнастика
801350556,online,mornig,Физическая активность,ОНЛАЙН ОФП
801350568,online,mornig,Физическая активность,ОНЛАЙН ОФП
801361065,offline,mornig,Рисование,Рисование


User recommendations:


,уникальный номер,направление 1,направление 2,направление 3,округ площадки,район площадки
1076,801349915,Физическая активность,ОНЛАЙН ОФП,ОНЛАЙН ОФП,"Северо-Восточный административный округ, Север...","муниципальный округ Марьина Роща, муниципальны..."
1613,801348774,Физическая активность,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,"Юго-Западный административный округ, Юго-Восто...","муниципальный округ Южное Бутово, муниципальны..."
1784,801348369,Физическая активность,ОНЛАЙН Гимнастика,ОНЛАЙН Гимнастика,Восточный административный округ,муниципальный округ Северное Измайлово
2897,801346717,Физическая активность,ОНЛАЙН Гимнастика,ОНЛАЙН Дыхательная гимнастика,NaN,NaN
2899,801346566,Физическая активность,ОНЛАЙН Гимнастика,ОНЛАЙН Дыхательная гимнастика,NaN,NaN
2909,801347778,Физическая активность,ОНЛАЙН Гимнастика,ОНЛАЙН Дыхательная гимнастика,NaN,NaN
2911,801348554,Физическая активность,ОНЛАЙН Гимнастика,ОНЛАЙН Дыхательная гимнастика,NaN,NaN
3730,801346573,Физическая активность,ОНЛАЙН ОФП,ОНЛАЙН ОФП,"Северный административный округ, Северный адми...","муниципальный округ Дмитровский, муниципальный..."
11484,801352857,Физическая активность,Гимнастика,Дыхательная гимнастика по системе Стрельниковой,Северо-Восточный административный округ,муниципальный округ Ростокино
23659,801373704,Центры московского долголетия,Свободное посещение,Свободное посещение,Восточный административный округ,муниципальный округ Сокольники


In [10]:
user_id = 101385865

display_history_and_predictions_for_user(user_id, actions, default_recommendations)

----------------------------------------------------------------------------
User 101385865 history:


group_id,is_online,session,направление 1,направление 2
801346923,online,mornig,Танцы,ОНЛАЙН Танцы
801348309,online,mornig,Творчество,ОНЛАЙН Красота и стиль
801348946,online,mornig,Физическая активность,ОНЛАЙН Гимнастика
801349093,online,mornig,Образование,ОНЛАЙН Иностранные языки
801350488,offline,aftennoon,Физическая активность,"Фитнес, тренажеры"
801350739,offline,mornig,Танцы,Танцы
801350759,offline,mornig,Танцы,Танцы
801351106,online,mornig,Творчество,ОНЛАЙН Красота и стиль
801351519,offline,mornig,Танцы,Танцы
801351888,offline,mornig,Танцы,Танцы


User recommendations:


,уникальный номер,направление 1,направление 2,направление 3,округ площадки,район площадки
1307,801347350,Физическая активность,ОНЛАЙН Гимнастика,ОНЛАЙН Корригирующая гимнастика для глаз,Восточный административный округ,муниципальный округ Северное Измайлово
1744,801350068,Творчество,ОНЛАЙН Красота и стиль,ОНЛАЙН Мастер-класс по уходу за кожей в зрелом...,"Юго-Западный административный округ,","муниципальный округ Ясенево,"
2144,801346784,Образование,"ОНЛАЙН История, искусство, краеведение","ОНЛАЙН История, культура России",NaN,NaN
2527,801352390,Физическая активность,ОНЛАЙН Гимнастика,ОНЛАЙН Корригирующая гимнастика для глаз,NaN,NaN
3557,801346557,Образование,ОНЛАЙН Психология и коммуникации,ОНЛАЙН Психологические тренинги,NaN,NaN
3752,801347074,Физическая активность,ОНЛАЙН Гимнастика,ОНЛАЙН Корригирующая гимнастика для глаз,NaN,NaN
3788,801347329,Физическая активность,ОНЛАЙН Гимнастика,ОНЛАЙН Корригирующая гимнастика для глаз,Восточный административный округ,муниципальный округ Северное Измайлово
9810,801351000,Физическая активность,ОНЛАЙН Гимнастика,ОНЛАЙН Корригирующая гимнастика для глаз,NaN,NaN
9812,801348614,Физическая активность,ОНЛАЙН Гимнастика,ОНЛАЙН Корригирующая гимнастика для глаз,NaN,NaN
11156,801349550,Физическая активность,ОНЛАЙН Гимнастика,ОНЛАЙН Корригирующая гимнастика для глаз,NaN,NaN


### Делаем сравнение различных моделей

Описание всех моделей, заложенных в turicreate, доступно по ссылке:

https://apple.github.io/turicreate/docs/api/turicreate.toolkits.recommender.html

In [11]:
print('⬛⬛⬛⬛⬛⬛⬛---------------\nTrain default model')
model_0 = tc.recommender.create(training_data, 'user_id', 'group_id', target = 'log_visits',
                                user_data = users_1, item_data = groups_1)

print('⬛⬛⬛⬛⬛⬛⬛---------------\nTrain item_similarity model 1')
model_1 = tc.item_similarity_recommender.create(training_data, 'user_id', 'group_id', target = 'log_visits',
                                                user_data = users_1, item_data = groups_1)

print('⬛⬛⬛⬛⬛⬛⬛---------------\nTrain item_similarity model 2')
model_1 = tc.item_similarity_recommender.create(training_data, 'user_id', 'group_id',
                                                user_data = users_1, item_data = groups_1)


print('⬛⬛⬛⬛⬛⬛⬛---------------\nTrain popularity_recommender model')
model_2 = tc.popularity_recommender.create(training_data, 'user_id', 'group_id', target = 'log_visits',
                                           user_data = users_1, item_data = groups_1)

print('⬛⬛⬛⬛⬛⬛⬛---------------\nTrain factorization_recommender model')
model_3 = tc.factorization_recommender.create(training_data, 'user_id', 'group_id', target = 'log_visits',
                                              user_data = users_1, item_data = groups_1)

print('⬛⬛⬛⬛⬛⬛⬛---------------\nTrain ranking_factorization_recommender model')
model_4 = tc.ranking_factorization_recommender.create(training_data, 'user_id', 'group_id',
                                                      user_data = users_1, item_data = groups_1)

⬛⬛⬛⬛⬛⬛⬛---------------
Train default model


Preparing data set.

Data has 600249 observations with 50906 users and 24535 items.

Data prepared in: 0.74489s

Training ranking_factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training Predictive Error.

Final objective value: 0.205477

Final training Predictive Error: 0.0607248

In [12]:
tc.recommender.util.compare_models(
    validation_data, 
    [model_0, model_1,model_2,model_3,model_4], 
    model_names=["⬛⬛⬛⬛⬛⬛⬛ Default model",
                 "⬛⬛⬛⬛⬛⬛⬛ Item similarity model",
                 '⬛⬛⬛⬛⬛⬛⬛ Popularity recommender',
                 '⬛⬛⬛⬛⬛⬛⬛ Factorization recommender',
                 '⬛⬛⬛⬛⬛⬛⬛ Ranking factorization recommender'])

PROGRESS: Evaluate model ⬛⬛⬛⬛⬛⬛⬛ Default model

Precision and recall summary statistics by cutoff
+--------+----------------+-------------+
| cutoff | mean_precision | mean_recall |
+--------+----------------+-------------+
|   1    |      0.0       |     0.0     |
|   2    |      0.0       |     0.0     |
|   3    |      0.0       |     0.0     |
|   4    |      0.0       |     0.0     |
|   5    |      0.0       |     0.0     |
|   6    |      0.0       |     0.0     |
|   7    |      0.0       |     0.0     |
|   8    |      0.0       |     0.0     |
|   9    |      0.0       |     0.0     |
|   10   |      0.0       |     0.0     |
+--------+----------------+-------------+
[10 rows x 3 columns]




Overall RMSE: 0.9807509470753443

Per User RMSE (best)
+-----------+---------------------+-------+
|  user_id  |         rmse        | count |
+-----------+---------------------+-------+
| 101410303 | 0.03251218011175805 |   1   |
+-----------+---------------------+-------+
[1 rows x 3 columns]


Per User RMSE (worst)
+-----------+-------------------+-------+
|  user_id  |        rmse       | count |
+-----------+-------------------+-------+
| 101386423 | 3.172759626878093 |   1   |
+-----------+-------------------+-------+
[1 rows x 3 columns]


Per Item RMSE (best)
+-----------+---------------------+-------+
|  group_id |         rmse        | count |
+-----------+---------------------+-------+
| 801366801 | 0.02801711718737221 |   1   |
+-----------+---------------------+-------+
[1 rows x 3 columns]


Per Item RMSE (worst)
+-----------+-------------------+-------+
|  group_id |        rmse       | count |
+-----------+-------------------+-------+
| 801348843 | 3.172759626878093 | 


Precision and recall summary statistics by cutoff
+--------+-----------------------+----------------------+
| cutoff |     mean_precision    |     mean_recall      |
+--------+-----------------------+----------------------+
|   1    |          0.0          |         0.0          |
|   2    |          0.0          |         0.0          |
|   3    |          0.0          |         0.0          |
|   4    |          0.0          |         0.0          |
|   5    |          0.0          |         0.0          |
|   6    |          0.0          |         0.0          |
|   7    | 0.0011904761904761904 | 0.008333333333333335 |
|   8    | 0.0010416666666666669 | 0.008333333333333335 |
|   9    |  0.001851851851851851 | 0.010416666666666668 |
|   10   | 0.0016666666666666668 | 0.010416666666666668 |
+--------+-----------------------+----------------------+
[10 rows x 3 columns]



[{'precision_recall_by_user': Columns:
  	user_id	int
  	cutoff	int
  	precision	float
  	recall	float
  	count	int
  
  Rows: 2160
  
  Data:
  +-----------+--------+-----------+--------+-------+
  |  user_id  | cutoff | precision | recall | count |
  +-----------+--------+-----------+--------+-------+
  | 101373685 |   1    |    0.0    |  0.0   |   1   |
  | 101373685 |   2    |    0.0    |  0.0   |   1   |
  | 101373685 |   3    |    0.0    |  0.0   |   1   |
  | 101373685 |   4    |    0.0    |  0.0   |   1   |
  | 101373685 |   5    |    0.0    |  0.0   |   1   |
  | 101373685 |   6    |    0.0    |  0.0   |   1   |
  | 101373685 |   7    |    0.0    |  0.0   |   1   |
  | 101373685 |   8    |    0.0    |  0.0   |   1   |
  | 101373685 |   9    |    0.0    |  0.0   |   1   |
  | 101373685 |   10   |    0.0    |  0.0   |   1   |
  +-----------+--------+-----------+--------+-------+
  [2160 rows x 5 columns]
  Note: Only the head of the SFrame is printed.
  You can use print_rows(nu

`Default model:             `Overall RMSE: 1.06 -> 1.01

`Item similarity model:     `Overall RMSE: 1.93 -> 1.98

`Popularity recommender:    `Overall RMSE: 0.98 -> 0.99

`Factorization recommender: `Overall RMSE: 1.00 -> 1.02

`Ranking factorization:     `Overall RMSE: 1.16 -> 0.97

### Сохраняем результаты Ranking factirozation модели

In [13]:
preds = model_4.recommend(k=10)
display(preds)

recommendations finished on 1000/50906 queries. users per second: 910.811

recommendations finished on 2000/50906 queries. users per second: 917.832

recommendations finished on 3000/50906 queries. users per second: 917.308

recommendations finished on 4000/50906 queries. users per second: 916.97

recommendations finished on 5000/50906 queries. users per second: 918.188

...

recommendations finished on 47000/50906 queries. users per second: 918.95

recommendations finished on 48000/50906 queries. users per second: 919.189

recommendations finished on 49000/50906 queries. users per second: 919.395

recommendations finished on 50000/50906 queries. users per second: 919.717

user_id,group_id,score,rank
101354797,801353793,0.8232877948893121,1
101354797,801353285,0.8006826386368828,2
101354797,801355955,0.7825871354328703,3
101354797,801353290,0.7817690849324592,4
101354797,801353281,0.7737643818273108,5
101354797,801349127,0.7666385081690917,6
101354797,801349305,0.7456999964208675,7
101354797,801358792,0.7050070735630181,8
101354797,801348527,0.7040772802803721,9
101354797,801353708,0.6707442089823471,10


### Сохраняем результаты для бэка

In [ ]:
preds.to_dataframe().to_csv('predicts/predict_v01.csv', index=False )

### Делаем предсказание для Test (с выборкой предсказаний на март-апрель 2023)

In [ ]:
import json
with open('groups2023.json') as json_file:
    groups2023 = json.load(json_file)

df_test = pd.read_csv('test.csv')
df_test.head()
df_test['уникальный номер участника'] = df_test['уникальный номер участника'].astype(int)


def display_history_and_predictions_for_user2(user_id, train_dataset, predictions):
    print('----------------------------------------------------------------------------')
    print('User', user_id, 'history:')
    #display(train_dataset[train_dataset['user_id'] == user_id]['group_id','is_online','session','направление 1','направление 2'].to_dataframe())
    display(train_dataset[train_dataset['user_id'] == user_id].to_dataframe())
    predictions = predictions[predictions['user_id'] == user_id]
    print('User recommendations:')
    predicted_groups = predictions['group_id'].to_list()
    display(df_groups[df_groups['уникальный номер'].isin(predicted_groups)].drop(columns=[
        'адрес площадки',
        'расписание в активных периодах',
        'расписание в закрытых периодах',
        'расписание в плановом периоде']))

In [ ]:
results = [] 


def get_groups_mix(user_id):
    # Predictions
    x0 = model_0.recommend(users=[user_id], k=1000).to_dataframe()
    x0 = x0[x0['group_id'].isin(groups2023)]
    x0 = x0.head(10)
    
    x1 = model_1.recommend(users=[user_id], k=1000).to_dataframe()
    x1 = x1[x1['group_id'].isin(groups2023)]
    x1 = x1.head(5)
    
    x2 = model_2.recommend(users=[user_id], k=1000).to_dataframe()
    x2 = x2[x2['group_id'].isin(groups2023)]
    x2 = x2.head(10)
    
    x3 = model_3.recommend(users=[user_id], k=1000).to_dataframe()
    x3 = x3[x3['group_id'].isin(groups2023)]
    x3 = x3.head(1)
    
    x4 = model_4.recommend(users=[user_id], k=1000).to_dataframe()
    x4 = x4[x4['group_id'].isin(groups2023)]
    x4 = x4.head(2)
    
    # Отбираем 10 штук, чтобы 1 была от 3 алгоритма и 2 от 4 алгоритма
    x = pd.concat([x1, x4, x0])
    x = x.drop_duplicates(subset='group_id').head(10) 
    
    return x
    
       

for index, row in df_test.iterrows():
    # Получаем юзера
    user_id = row['уникальный номер участника']
    
    x = get_groups_mix(user_id)
    
    display_history_and_predictions_for_user2(user_id, training_data, x)

    groups_list = x['group_id'].to_list()
    groups_list = [str(x) for x in groups_list]    
    groups_str = ','.join(groups_list)
    results.append(groups_str)
 

In [ ]:
df_test['уникальный номер группы'] = results
df_test.to_csv('test.csv', index=False)

In [ ]:
display(df_test)